In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

In [5]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
X_train_processing = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=25, # 0~25도 사이에서 임의의 각도로 원본이미지를 회전
                                   width_shift_range=0.05, # 0.05범위 내에서 임의의 값만큼 임의의 방향으로 좌우 이동
                                   height_shift_range=0.05, # 0.05범위 내에서 임의의 값만큼 임의의 방향으로 상하 이동
                                   zoom_range=0.2, # (1-0.2)~(1+0.2) => 0.8~1.2 사이에서 임의의 수치만큼 확대/축소
                                   horizontal_flip=True, # 좌우로 뒤집기                                   
                                   vertical_flip=True,
                                   fill_mode='nearest'
                                  ) 
X_validation_processing = ImageDataGenerator(rescale = 1./255)
X_test_processing = ImageDataGenerator(rescale = 1./255)

In [7]:
#w/o poisoning
#train_dir = './xray_undersampling/train/'
#test_dir = './xray_undersampling/test/'

In [8]:
#poisoning dataset
train_dir = './50%/train/'
test_dir = './50%/test/'

In [9]:
X_train = X_train_processing.flow_from_directory(train_dir, 
                                                    batch_size=16, # 한번에 변환된 이미지 16개씩 만들어라 라는 것
                                                    color_mode='grayscale', # grayscale: 흑백 이미지 처리
                                                    class_mode='binary', 
                                                    target_size=(150,150)) # target_size에 맞춰서 이미지의 크기가 조절된다

X_test = X_test_processing.flow_from_directory(test_dir,
                                                  batch_size=4,
                                                  color_mode='grayscale',
                                                  class_mode='binary',
                                                  target_size=(150,150))

Found 2682 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [10]:
X_train.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [11]:
import keras
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential

from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation

In [12]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")

model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[150,150, 1]), #커널 7x7, 필터 64개
    keras.layers.MaxPooling2D(pool_size=2), #크기 1/2로 줄어듦
    DefaultConv2D(filters=128), #나머지는 default로
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(), #중간에 flatten
    keras.layers.Dense(units=128, activation='relu'), 
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=1, activation='sigmoid'),
])

In [13]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0),
    optimizer=RMSprop(lr=1e-4),
    metrics=['acc']
)


/home/agent/.local/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [14]:
history = model.fit_generator(
    X_train,
    steps_per_epoch=10,
    epochs=30,
    validation_steps=50
)

<ipython-input-14-bbda1bd4caff>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30
10/10 [==============================] - 18s 1s/step - loss: 0.7080 - acc: 0.4812
Epoch 2/30
10/10 [==============================] - 13s 1s/step - loss: 0.6942 - acc: 0.5562
Epoch 3/30
10/10 [==============================] - 13s 1s/step - loss: 0.6970 - acc: 0.4625
Epoch 4/30
10/10 [==============================] - 13s 1s/step - loss: 0.6967 - acc: 0.5000
Epoch 5/30
10/10 [==============================] - 12s 1s/step - loss: 0.6944 - acc: 0.4437
Epoch 6/30
10/10 [==============================] - 12s 1s/step - loss: 0.6943 - acc: 0.4250
Epoch 7/30
10/10 [==============================] - 12s 1s/step - loss: 0.6933 - acc: 0.4875
Epoch 8/30
10/10 [==============================] - 12s 1s/step - loss: 0.6920 - acc: 0.5250
Epoch 9/30
10/10 [==============================] - 12s 1s/step - loss: 0.6955 - acc: 0.4563
Epoch 10/30
10/10 [==============================] - 12s 1s/step - loss: 0.6938 - acc: 0.5375
Epoch 11/30
10/10 [==============================] - 12s 1s/step - lo

In [15]:
# 모델 성능 평가
model.evaluate(X_test)

156/156 [==============================] - 11s 71ms/step - loss: 0.6925 - acc: 0.6250


[0.6924883723258972, 0.625]

In [16]:
model.compile(
    loss=tf.keras.losses.BinaryFocalCrossentropy(),
    optimizer=RMSprop(lr=1e-4),
    metrics=['acc']
)


In [17]:
history = model.fit_generator(
    X_train,
    steps_per_epoch=10, 
    epochs=30,
    validation_steps=50
)

Epoch 1/30


<ipython-input-17-50da17d6b332>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


10/10 [==============================] - 13s 1s/step - loss: 0.1740 - acc: 0.4875
Epoch 2/30
10/10 [==============================] - 11s 1s/step - loss: 0.1734 - acc: 0.4805
Epoch 3/30
10/10 [==============================] - 12s 1s/step - loss: 0.1735 - acc: 0.4750
Epoch 4/30
10/10 [==============================] - 12s 1s/step - loss: 0.1734 - acc: 0.4500
Epoch 5/30
10/10 [==============================] - 12s 1s/step - loss: 0.1727 - acc: 0.5063
Epoch 6/30
10/10 [==============================] - 12s 1s/step - loss: 0.1739 - acc: 0.4938
Epoch 7/30
10/10 [==============================] - 12s 1s/step - loss: 0.1739 - acc: 0.4688
Epoch 8/30
10/10 [==============================] - 11s 1s/step - loss: 0.1730 - acc: 0.5437
Epoch 9/30
10/10 [==============================] - 12s 1s/step - loss: 0.1727 - acc: 0.5813
Epoch 10/30
10/10 [==============================] - 11s 1s/step - loss: 0.1724 - acc: 0.5750
Epoch 11/30
10/10 [==============================] - 12s 1s/step - loss: 0.1737 

In [18]:
# 모델 성능 평가
model.evaluate(X_test)

156/156 [==============================] - 12s 78ms/step - loss: 0.1702 - acc: 0.6250


[0.17023253440856934, 0.625]